In [8]:
import requests
import pickle
import pandas as pd
import pymysql
import json
import re

### 룬 관련 정보 URL 호출

In [9]:
url = 'https://ddragon.leagueoflegends.com/cdn/12.19.1/data/ko_KR/runesReforged.json'
datas = requests.get(url)
datas = datas.json()

### 데이터 추출

In [16]:
def rename(st):
    txt = re.compile('(<([^>]+)>)')
    result = txt.sub('',st)
    return result

In [17]:
runes = []
for data in datas:
    runes.append([data['id'],data['name'],'https://ddragon.leagueoflegends.com/cdn/img/'+data['icon'],0,0])
    for rune in data['slots']:
        for r in rune['runes']:
            runes.append([r['id'],r['name'],'https://ddragon.leagueoflegends.com/cdn/img/'+r['icon'],rename(r['shortDesc']),rename(r['longDesc'])])
rune = pd.DataFrame(runes, columns=['id','name','icon','shortDesc','longDesc'])
rune.head()

,id,name,icon,shortDesc,longDesc
0,8100,지배,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0
1,8112,감전,https://ddragon.leagueoflegends.com/cdn/img/pe...,3초 동안 같은 챔피언에게 기본 공격 또는 개별 스킬 3회를 적중시키면 추가 적응형...,3초 동안 같은 챔피언에게 개별 공격 또는 스킬을 3회 적중시키면 추가 적응형 피해...
2,8124,포식자,https://ddragon.leagueoflegends.com/cdn/img/pe...,장화에 사용 효과 추가. 사용 시 일시적으로 이동 속도가 대폭 증가하며 다음 공격 ...,장화에 '포식자' 사용 효과를 부여합니다.적 챔피언을 쫓을 때 이동 속도가 1초 동...
3,8128,어둠의 수확,https://ddragon.leagueoflegends.com/cdn/img/pe...,체력이 낮은 챔피언에게 피해를 입히면 적응형 피해를 입히고 해당 챔피언의 영혼을 수확,체력이 50%보다 낮은 챔피언에게 피해를 입히면 적응형 피해를 추가로 입히고 해당 ...
4,9923,칼날비,https://ddragon.leagueoflegends.com/cdn/img/pe...,적 챔피언에 대한 첫 공격 3회 동안 공격 속도 대폭 증가,적 챔피언에 대한 3번째 공격까지 공격 속도가 110% 증가합니다.3초 안에 다음 ...


### 해당 데이터 프레임 pickle에 저장

In [20]:
with open('rune.pickle','wb') as f:
    pickle.dump(rune,f)

### 데이터 확인

In [47]:
rune[rune['shortDesc']==0]

,id,name,icon,shortDesc,longDesc
0,8100,지배,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0
15,8300,영감,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0
28,8000,정밀,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0
42,8400,결의,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0
55,8200,마법,https://ddragon.leagueoflegends.com/cdn/img/pe...,0,0


In [40]:
rune[rune['shortDesc']!=0]

,id,name,icon,shortDesc,longDesc
1,8112,감전,https://ddragon.leagueoflegends.com/cdn/img/pe...,3초 동안 같은 챔피언에게 기본 공격 또는 <b>개별</b> 스킬 3회를 적중시키면...,3초 동안 같은 챔피언에게 <b>개별</b> 공격 또는 스킬을 3회 적중시키면 추가...
2,8124,포식자,https://ddragon.leagueoflegends.com/cdn/img/pe...,장화에 사용 효과 추가. 사용 시 일시적으로 <lol-uikit-tooltipped...,장화에 '<font color='#c60300'>포식자</font>' 사용 효과를 ...
3,8128,어둠의 수확,https://ddragon.leagueoflegends.com/cdn/img/pe...,체력이 낮은 챔피언에게 피해를 입히면 <lol-uikit-tooltipped-key...,체력이 50%보다 낮은 챔피언에게 피해를 입히면 <lol-uikit-tooltipp...
4,9923,칼날비,https://ddragon.leagueoflegends.com/cdn/img/pe...,적 챔피언에 대한 첫 공격 3회 동안 공격 속도 대폭 증가,적 챔피언에 대한 3번째 공격까지 공격 속도가 110% 증가합니다.<br><br>3...
5,8126,비열한 한 방,https://ddragon.leagueoflegends.com/cdn/img/pe...,<lol-uikit-tooltipped-keyword key='LinkTooltip...,<b>이동 또는 행동을 방해받은</b> 상태의 챔피언에게 피해를 주면 레벨에 따라 ...
...,...,...,...,...,...
63,8234,기민함,https://ddragon.leagueoflegends.com/cdn/img/pe...,모든 추가 <speed>이동 속도</speed> 효과가 7% 증가하고 <speed>...,모든 추가 이동 속도 효과가 7% 증가하고 <speed>이동 속도를 1%</spee...
64,8233,절대 집중,https://ddragon.leagueoflegends.com/cdn/img/pe...,체력이 70% 이상일 경우 <lol-uikit-tooltipped-keyword k...,체력이 70% 이상일 경우 레벨에 따라 <lol-uikit-tooltipped-ke...
65,8237,주문 작열,https://ddragon.leagueoflegends.com/cdn/img/pe...,10초마다 공격 스킬 적중 시 챔피언을 불태움,다음 공격 스킬 적중 시 챔피언에게 불을 붙여 1초 후 레벨에 따라 20~40의 추...
66,8232,물 위를 걷는 자,https://ddragon.leagueoflegends.com/cdn/img/pe...,강에 있을 때 <lol-uikit-tooltipped-keyword key='Lin...,강에 있을 때 <speed>이동 속도가 25</speed> 증가하고 레벨에 비례하여...


### DB에 데이터 저장

In [21]:
runeId = rune['id'].values
name = rune['name'].values
icon = rune['icon'].values
shortDesc = rune['shortDesc'].values
longDesc = rune['longDesc'].values

In [22]:
conn = pymysql.connect(host='3.36.255.55', user='root', password='root1234', db='lol_project', charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)
for a,b,c,d,e in zip(runeId,name,icon,shortDesc,longDesc):
    sql = "insert into rune (runeId,name,icon,shortDesc,longDesc) values (%s,%s,%s,%s,%s)"
    val = (a,b,c,d,e)
    cur.execute(sql,val)
conn.commit()
conn.close()